In [10]:
import cv2
import os
from skimage import filters, feature
import skimage
import numpy as np
import matplotlib.pyplot as plt

In [11]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [12]:
cushion = load_images_from_folder("Diamonds2/images/cushion")
cushion = np.array(cushion)
cushion.shape

NameError: name 'os' is not defined

In [ ]:
emerald = load_images_from_folder("Diamonds2/images/emerald")
emerald = np.array(emerald)
emerald.shape

In [ ]:
heart = load_images_from_folder("Diamonds2/images/heart")
heart = np.array(heart)
heart.shape

In [ ]:
marquise = load_images_from_folder("Diamonds2/images/marquise")
marquise = np.array(marquise)
marquise.shape

In [ ]:
oval = load_images_from_folder("Diamonds2/images/oval")
oval = np.array(oval)
oval.shape

In [ ]:
pear = load_images_from_folder("Diamonds2/images/pear")
pear = np.array(pear)
pear.shape

In [ ]:
princess = load_images_from_folder("Diamonds2/images/princess")
princess = np.array(princess)
princess.shape

In [ ]:
round = load_images_from_folder("Diamonds2/images/round")
round = np.array(round)
round.shape

In [ ]:
import pandas as pd
import numpy as np
import os
# E.g., cuishons directory and corresponding csv for cuishons 
# Returns comma-separated values (csv) file is returned as two-dimensional data structure with labeled axes.
def extract_data(stone_directory, stone_csv):
    csv_data = np.asarray(pd.read_csv(stone_csv))

    # parallel arrays with corresponding indices
    clarity = []
    id = []
    image_directory = []
    for row in range(len(csv_data)):

        stone_directory
        for root, _, files in os.walk(stone_directory):
            for name in files:
                if name.split(".")[0]==(str(csv_data[row][0])):
                    image_directory.append(os.path.join(root, name))
                    id.append(csv_data[row][0])
                    clarity.append(csv_data[row][3])

    return image_directory, clarity, id

In [ ]:
cushion_images, cushion_clarity, cushion_id = extract_data("Diamonds2/images/cushion", "Diamonds2/data_cushion.csv")

In [ ]:
emerald_images, emerald_clarity, emerald_id = extract_data("Diamonds2/images/emerald", "Diamonds2/data_emerald.csv")

In [ ]:
heart_images, heart_clarity, heart_id = extract_data("Diamonds2/images/heart", "Diamonds2/data_heart.csv")

In [ ]:
marquise_images, marquise_clarity, marquise_id = extract_data("Diamonds2/images/marquise", "Diamonds2/data_marquise.csv")

In [ ]:
oval_images, oval_clarity, oval_id = extract_data("Diamonds2/images/oval", "Diamonds2/data_oval.csv")

In [ ]:
pear_images, pear_clarity, pear_id = extract_data("Diamonds2/images/pear", "Diamonds2/data_pear.csv")

In [ ]:
princess_images, princess_clarity, princess_id = extract_data("Diamonds2/images/princess", "Diamonds2/data_princess.csv")

In [ ]:
round_images, round_clarity, round_id = extract_data("Diamonds2/images/round", "Diamonds2/data_round.csv")

In [ ]:
def load_images(path):
    images = []
    for file in path:
        img = cv2.imread(file)
        if img is not None:
            images.append(img)
    return images 

In [ ]:
marquise2 = load_images(marquise_images)
marquise2 = np.array(marquise2)
marquise2.shape

In [ ]:
def remove_background(filepath):
    img = cv2.imread(filepath)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    mask = cv2.threshold(gray, (gray[0][0] - 10), gray[0][0] + 10, cv2.THRESH_BINARY_INV)[1]

    mask = cv2.GaussianBlur(mask, (0,0), sigmaX=1, borderType=cv2.BORDER_DEFAULT)

    result = img.copy()
    result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
    result[:, :, 3] = mask

    alpha = np.asarray(mask, dtype='float32')/255.0

    #White Background - Corresponds to figure 8 in paper
    result[:,:,0] = result[:,:,0] * (alpha)
    result[:,:,1] = result[:,:,1] * (alpha)
    result[:,:,2] = result[:,:,2] * (alpha)

    result = result[:,:,0:3]

    return result[:,:,0]

In [ ]:
plt.imshow(cv2.imread(round_images[68]))

In [ ]:
plt.imshow(cv2.imread(round_images[71]))

In [ ]:
def get_feature_points(image, feature_width):
    # Note that xs and ys represent the coordinates of the image. Thus, xs actually denote the columns
    # of the respective points and ys denote the rows of the respective points.

    #Calculate the gradient (partial derivatives on two directions).
    grad_h = skimage.filters.sobel_h(image)
    grad_v = skimage.filters.sobel_v(image)
    multiplied_components = np.multiply(grad_v, grad_h)
    grad_v_squared = np.square(grad_v) #comp_hh_m
    grad_h_squared = np.square(grad_h) #comp_ww_m

    #Gaussian Filter
    gaussian_grad_v_squared = skimage.filters.gaussian(grad_v_squared) #gass_hh_m
    gaussian_grad_h_squared = skimage.filters.gaussian(grad_h_squared) #gass_ww_m
    gaussian_multiplied_components = skimage.filters.gaussian(multiplied_components) #gass_hw_m

    #
    det = np.subtract(np.multiply(gaussian_grad_v_squared, gaussian_grad_h_squared), np.square(gaussian_multiplied_components))
    trace = np.add(gaussian_grad_v_squared, gaussian_grad_h_squared)
    c = np.subtract(det, 0.035*np.square(trace))
    mean = np.mean(c)
    # STEP 4: Peak local max to eliminate clusters. (Try different parameters.)
    peak_local_max = skimage.feature.peak_local_max(c, feature_width, threshold_abs=mean, indices=True)

    xs = peak_local_max[:, 1]

    ys = peak_local_max[:, 0]

    return xs, ys

In [ ]:
Diamond_id = 68
fix_image = remove_background(round_images[Diamond_id])
x, y = get_feature_points(fix_image, 2)

In [ ]:
def plot_interest_points(image, x, y):
    my_dpi = 120
    fig = plt.figure(figsize=(390/my_dpi, 390/my_dpi), dpi=my_dpi, frameon=False)
    plt.axis('off')
    plt.scatter(x,y, color=(1,0,0), s=10)
    plt.imshow(image, cmap="gray")

    fig.savefig("Test", bbox_inches='tight', pad_inches=0)

In [ ]:
plot_interest_points(fix_image, x, y)

In [ ]:
def save_interest_points_cushion(image, x, y, id):
    my_dpi = 120
    fig = plt.figure(figsize=(390/my_dpi, 390/my_dpi), dpi=my_dpi, frameon=False)
    plt.axis('off')
    plt.scatter(x,y, color=(1,0,0), s=10)
    plt.imshow(image, cmap="gray")

    fig.savefig("Diamonds2/images_harris_2/cushion/" + cushion_id[id], bbox_inches='tight', pad_inches=0)
    plt.clf()
    fig.clf()

def save_interest_points_emerald(image, x, y, id):
    my_dpi = 120
    fig = plt.figure(figsize=(390/my_dpi, 390/my_dpi), dpi=my_dpi, frameon=False)
    plt.axis('off')
    plt.scatter(x,y, color=(1,0,0), s=10)
    plt.imshow(image, cmap="gray")

    fig.savefig("Diamonds2/images_harris_2/emerald/" + emerald_id[id], bbox_inches='tight', pad_inches=0)
    plt.clf()
    fig.clf()

def save_interest_points_heart(image, x, y, id):
    my_dpi = 120
    fig = plt.figure(figsize=(390/my_dpi, 390/my_dpi), dpi=my_dpi, frameon=False)
    plt.axis('off')
    plt.scatter(x,y, color=(1,0,0), s=10)
    plt.imshow(image, cmap="gray")

    fig.savefig("Diamonds2/images_harris_2/heart/" + str(heart_id[id]), bbox_inches='tight', pad_inches=0)
    plt.clf()
    fig.clf()

def save_interest_points_marquise(image, x, y, id):
    my_dpi = 120
    fig = plt.figure(figsize=(390/my_dpi, 390/my_dpi), dpi=my_dpi, frameon=False)
    plt.axis('off')
    plt.scatter(x,y, color=(1,0,0), s=10)
    plt.imshow(image, cmap="gray")

    fig.savefig("Diamonds2/images_harris_2/marquise/" + marquise_id[id], bbox_inches='tight', pad_inches=0)
    plt.clf()
    fig.clf()

def save_interest_points_oval(image, x, y, id):
    my_dpi = 120
    fig = plt.figure(figsize=(390/my_dpi, 390/my_dpi), dpi=my_dpi, frameon=False)
    plt.axis('off')
    plt.scatter(x,y, color=(1,0,0), s=10)
    plt.imshow(image, cmap="gray")

    fig.savefig("Diamonds2/images_harris_2/oval/" + oval_id[id], bbox_inches='tight', pad_inches=0)
    plt.clf()
    fig.clf()

def save_interest_points_pear(image, x, y, id):
    my_dpi = 120
    fig = plt.figure(figsize=(390/my_dpi, 390/my_dpi), dpi=my_dpi, frameon=False)
    plt.axis('off')
    plt.scatter(x,y, color=(1,0,0), s=10)
    plt.imshow(image, cmap="gray")

    fig.savefig("Diamonds2/images_harris_2/pear/" + pear_id[id], bbox_inches='tight', pad_inches=0)
    plt.clf()
    fig.clf()

def save_interest_points_princess(image, x, y, id):
    my_dpi = 120
    fig = plt.figure(figsize=(390/my_dpi, 390/my_dpi), dpi=my_dpi, frameon=False)
    plt.axis('off')
    plt.scatter(x,y, color=(1,0,0), s=10)
    plt.imshow(image, cmap="gray")

    fig.savefig("Diamonds2/images_harris_2/princess/" + princess_id[id], bbox_inches='tight', pad_inches=0)
    plt.clf()
    fig.clf()

def save_interest_points_round(image, x, y, id):
    my_dpi = 120
    fig = plt.figure(figsize=(390/my_dpi, 390/my_dpi), dpi=my_dpi, frameon=False)
    plt.axis('off')
    plt.scatter(x,y, color=(1,0,0), s=10)
    plt.imshow(image, cmap="gray")

    fig.savefig("Diamonds2/images_harris_2/round/" + round_id[id], bbox_inches='tight', pad_inches=0)
    plt.clf()
    fig.clf()

In [ ]:
for img in range(len(cushion_images)):
    fixed_image = remove_background(cushion_images[img])
    x, y = get_feature_points(fixed_image, 2)
    save_interest_points_cushion(fixed_image, x, y, img)
    x = []
    y = []

In [ ]:
for img in range(len(emerald_images)):

    fixed_image = remove_background(emerald_images[img])

    x, y = get_feature_points(fixed_image, 2)
    save_interest_points_emerald(fixed_image, x, y, img)
    x = []
    y = []

In [ ]:
for img in range(len(heart_images)):

    fixed_image = remove_background(heart_images[img])

    x, y = get_feature_points(fixed_image, 2)

    save_interest_points_heart(fixed_image, x, y, img)

    x = []
    y = []

In [ ]:
for img in range(len(marquise_images)):

    fixed_image = remove_background(marquise_images[img])

    x, y = get_feature_points(fixed_image, 2)

    save_interest_points_marquise(fixed_image, x, y, img)

    x = []
    y = []

In [ ]:
for img in range(len(oval_images)):

    fixed_image = remove_background(oval_images[img])

    x, y = get_feature_points(fixed_image, 2)

    save_interest_points_oval(fixed_image, x, y, img)

    x = []
    y = []

In [ ]:
for img in range(len(pear_images)):

    fixed_image = remove_background(pear_images[img])

    x, y = get_feature_points(fixed_image, 2)

    save_interest_points_pear(fixed_image, x, y, img)

    x = []
    y = []

In [ ]:
for img in range(len(princess_images)):

    fixed_image = remove_background(princess_images[img])

    x, y = get_feature_points(fixed_image, 2)

    save_interest_points_princess(fixed_image, x, y, img)

    x = []
    y = []

In [ ]:
for img in range(len(round_images)):

    fixed_image = remove_background(round_images[img])

    x, y = get_feature_points(fixed_image, 2)

    save_interest_points_round(fixed_image, x, y, img)

    x = []
    y = []